# Gestion de Usuario y Perfiles

In [ ]:
from pymongo import MongoClient
from datetime import datetime
from bson.objectid import ObjectId
import pandas as pd

In [ ]:
client = MongoClient("mongodb://localhost:27017")

In [ ]:
db = client["bdnr"]
users = db["users"]
print("Conectado a MongoDB correctamente.")

In [ ]:
users.drop()

In [ ]:
users.create_index("email", unique=True)

## Funcion para crear datos

In [ ]:
def create_user(username, email, profile_pic_url=None, account_status="Free"):
    user = {
        "username": username,
        "email": email,
        "profile_pic_url": profile_pic_url,
        "account_status": account_status,
        "stats": {
            "streak_days": 0,
            "current_league": None,
            "last_active": datetime.utcnow()
        },
        "learning_courses": [],
        "two_factor": {
            "enabled": False,
            "method": None
        },
        "friends_list": [],
        "achievements": [],
        "privacy_settings": {
            "is_profile_public": True,
            "can_receive_messages": True
        }
    }

    try:
        result = users.insert_one(user)
        print(f"Usuario {username} creado correctamente [userId: {result.inserted_id}].")
        return str(result.inserted_id)

    except Exception as e:
        if "duplicate key error" in str(e):
            print(f"Error: El email '{email}' ya está registrado.")
        else:
            print("Error inesperado:", e)
        return None

## Funciones de Lectura

In [ ]:
def get_user(user_id):
    return users.find_one({"_id": ObjectId(user_id)})

In [ ]:
def get_all_users():
    return users.find()

In [ ]:
def users_dataframe():
    data = get_all_users()
    if not data:
        return pd.DataFrame()
    return pd.DataFrame(data)


## Funciones para Actualizar los datos

In [ ]:
def update_streak(user_id, streak_days):
    users.update_one(
        {"_id": ObjectId(user_id)},
        {"$set": {
            "stats.streak_days": streak_days,
            "stats.last_active": datetime.utcnow()
        }}
    )
    print(f"Streak actualizado para userId {user_id}.")


In [ ]:
def update_privacy(user_id, **settings):
    users.update_one(
        {"_id": ObjectId(user_id)},
        {"$set": {f"privacy_settings.{k}": v for k, v in settings.items()}}
    )
    print(f"Privacidad actualizada para userId {user_id}.")


In [ ]:
def add_course(user_id, code, level):
    course = {"code": code, "level": level}
    
    users.update_one(
        {"_id": ObjectId(user_id)},
        {"$push": {"learning_courses": course}}
    )
    print(f"Curso agregado ({code}, nivel {level}) al usuario {user_id}.")

In [ ]:
def add_friend(user_id, friend_id):
    users.update_one(
        {"_id": ObjectId(user_id)},
        {"$push": {"friends_list": friend_id}}
    )
    print(f"Amigo {friend_id} agregado a {user_id}.")

In [ ]:
def add_achievement(user_id, achievement_code):
    users.update_one(
        {"_id": ObjectId(user_id)},
        {"$push": {"achievements": achievement_code}}
    )
    print(f"Logro '{achievement_code}' agregado a {user_id}.")

## Pruebas de con datos

### Crear usuarios

In [ ]:
id1 = create_user("josefina", "josefina@gmail.com")
id2 = create_user("matias", "matias@gmail.com", "fotoPerfil.jpg", "Plus")
id3 = create_user("juan pedro", "juanpedro@gmail.com")

In [ ]:
create_user("josefina", "josefina@gmail.com")

### Ver datos

In [ ]:
users_dataframe()

In [ ]:
get_user(id1)

### Actualizar datos

#### Actualizar racha

In [ ]:
update_streak(id1, 2)

In [ ]:
get_user(id1)['stats']['streak_days']

#### Actualizar privacidad

In [ ]:
update_privacy(id1, is_profile_public=False, can_receive_messages=False)
get_user(id1)['privacy_settings']

In [ ]:
update_privacy(id2, can_receive_messages=False)
get_user(id2)['privacy_settings']

#### Actualizar cursos

In [ ]:
add_course(id3, "en", 15)
get_user(id3)['learning_courses']

In [ ]:
add_course(id3, "fr", 2)
get_user(id3)['learning_courses']

#### Actualizar amigos

In [ ]:
add_friend(id1, id2)

In [ ]:
get_user(id1)['friends_list']

In [ ]:
get_user(id2)['friends_list']

In [ ]:
add_friend(id2, id1)

In [ ]:
get_user(id2)['friends_list']

#### Actualizar logros

In [ ]:
add_achievement(id3, "Language_Master_EN")
get_user(id3)

#### Ver todos los cambios

In [ ]:
users_dataframe()

## Agregar nueva Funcionalidad

In [ ]:
def set_daily_xp_goal(user_id, xp_goal):
    users.update_one(
        {"_id": ObjectId(user_id)},
        {"$set": {"daily_xp_goal": xp_goal}}
    )
    print(f"Meta diaria XP = {xp_goal} asignada al usuario {user_id}.")

In [ ]:
set_daily_xp_goal(id1, 30)  
get_user(id1)

In [ ]:
get_user(id2)

In [ ]:
users_dataframe()